<a href="https://colab.research.google.com/github/thimotyb/spark-notebooks/blob/main/mlflow_quickstart_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Compare runs, choose a model, and deploy it to a REST API
- Run a hyperparameter sweep on a training script
- Compare the results of the runs in the MLflow UI
- Choose the best run and register it as a model
- Deploy the model to a REST API
- Build a container image suitable for deployment to a cloud platform


https://mlflow.org/docs/latest/getting-started/quickstart-2/index.html



In [ ]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"


In [ ]:
!pip install tensorflow
!pip install hyperopt

In [3]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip list

Package                            Version
---------------------------------- -----------
absl-py                            2.1.0
alembic                            1.13.3
asttokens                          2.4.1
astunparse                         1.6.3
blinker                            1.8.2
boltons                            23.0.0
brotlipy                           0.7.0
cachetools                         5.5.0
certifi                            2024.8.30
cffi                               1.15.1
charset-normalizer                 2.0.4
click                              8.1.7
cloudpickle                        3.1.0
comm                               0.2.2
conda                              23.9.0
conda-content-trust                0.2.0
conda-libmamba-solver              23.9.1
conda-package-handling             2.2.0
conda_package_streaming            0.9.0
contourpy                          1.3.0
cryptography                       41.0.3
cycler                             0.12

In [4]:
from tensorflow import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature


In [5]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)


In [6]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


In [7]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result



In [8]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}


In [9]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


2024/10/24 11:19:26 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


Epoch 1/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 59s 1s/step - loss: 36.9036 - root_mean_squared_error: 6.0748
10/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 35.8257 - root_mean_squared_error: 5.9850
29/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 33.8777 - root_mean_squared_error: 5.8186
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 32.3196 - root_mean_squared_error: 5.6810 - val_loss: 21.9794 - val_root_mean_squared_error: 4.6882

Epoch 2/3

 1/46 ━━━━━━━━━━━━━━━━━━━━ 20s 447ms/step - loss: 21.1512 - root_mean_squared_error: 4.5990
18/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 20.3730 - root_mean_squared_error: 4.5132   
31/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 19.6904 - root_mean_squared_error: 4.4361
32/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 19.6454 - root_mean_squared_error: 4.4310
41/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 19.2399 - root_mean_squared_error: 4.3844
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 18.9678 - root_mean_squared_error: 4.3527 - val_loss: 12.7479 - 